## Simple LLM Application with LCEL 

This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some promting. Still, this is a great way to get started with Langchain - A lot of features can be build with just some prompting and an LLM call. 

After this, you'll have a high level overview of?
- Using language models
- Using PromptTemplatesd and OutputParsers.
- Using Langchain expression language (LCEL) to chain componenets together
- Debgging and tracing your application using LangSmith 
- Deploying your application with LangServe

### Groq - For Opensource LLM model (Llamma, Gemma2, Mistral)
- Groq is fast AI inference. 
- Groq is an LPU (Language Processing Unit) engine. 
- LPU is a hardware and software platform that delivers exceptional compute speed, quality, and energy efficiently. 
- LPU is designed to overcome the 2 LLM bottlenecks
    1. Compute Density
    2. Memory Bandwidth
- LPU has greater compute capacity than a GPU and CPU in regards to LLM -> This reduces the amount of time per word calculated, allowing sequences of text to be generated much faster. 
- Also eliminates external memory bottlebeck which enables the LPU inference engine to deliver orders of magnitude better performance on LLM compated to GPUs.

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x109ae7650>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x109b18200>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="Translate the following from English to Hindi"),
    HumanMessage(content="Hello, how are you?")
]

result = model.invoke(messages)
result.content

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n'

In [7]:
## String Output Parser 
from langchain_core.output_parsers import StrOutputParser 
parser = StrOutputParser()
parser.invoke(result)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n'

In [8]:
## Using LCEL - to chain the components 
chain = model | parser # first call the mode and then parse the response
chain.invoke(messages)

'**नमस्ते, आप कैसे हैं?** (Namaste, aap kaise hain?) \n\n\nThis is a common and polite way to say "Hello, how are you?" in Hindi. \n\n'

In [9]:
## Prompt Templates 
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template), 
        ("user", "{text}")
    ]
)

In [11]:
prompt = chat_prompt.invoke({"language" : "Hindi", "text": "Hello, how are you?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Hindi:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [ ]:
## Chaining Components together with LCEL
chain = prompt | model | parser
chain.invoke({"language" : "Hindi", "text": "Hello, how are you?"})

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nThis is the most common and polite way to say "Hello, how are you?" in Hindi. \n'